In [ ]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
!pip install textblob

In [ ]:
from textblob import TextBlob
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_csv('dataset-pemilu.csv')
data

,full_text,label
0,pasti anies tidak punya prestasi,negatif
1,suasana desa memang selalu senang udara segar ...,netral
2,program anies hanya ubah nama rumah sakit se i...,negatif
3,anies bohong dan bohong tanpa rasa salah,negatif
4,tidak bicara sesuai fakta anies dengan lantang...,negatif
...,...,...
2456,kata sila cek di gugel udah banyak swasta yang...,netral
2457,yang penting bukan tugas partai apalagi bokeper,netral
2458,solo bukti bahwa gibran punya rencana ekonomi ...,positif
2459,tapi fans nya yang muslim pada tidak paling im...,netral


In [ ]:
# dataset = data
# dataset = [tuple(x) for x in dataset.to_records(index=False)]

In [ ]:
# import random

# set_positif = []
# set_negatif = []
# set_netral = []

# for n in dataset:
#   if(n[1] == 'positif'):
#     set_positif.append(n)
#   elif(n[1] == 'negatif'):
#     set_negatif.append(n)
#   else:
#     set_netral.append(n)
# set_positif = random.sample(set_positif, k=int(len(set_positif)*4/5))
# set_negatif = random.sample(set_negatif, k=int(len(set_negatif)*4/5))
# set_netral = random.sample(set_netral, k=int(len(set_netral)*4/5))

# train = set_positif + set_negatif + set_netral

# test = list(set(dataset) - set(train))

# # 20% of the test data
# test_set = random.sample(test, k=int(len(test)*1/5))

# train_set = []

# for n in train:
#   train_set.append(n)

In [ ]:
# train_data = pd.DataFrame(train_set, columns=["full_text", "label"])
# test_data = pd.DataFrame(test_set, columns=["full_text", "label"])

In [ ]:
train_data, test_data = train_test_split(data, random_state=42, test_size=0.2)

In [ ]:
train = [tuple(x) for x in train_data.to_records(index=False)]
test = [tuple(x) for x in test_data.to_records(index=False)]

In [ ]:
from textblob.classifiers import NaiveBayesClassifier
cl = NaiveBayesClassifier(train)
print('akurasi: ', cl.accuracy(dataset))

akurasi:  0.819585534335636


In [ ]:
#labelling
data_tweet = list(data['full_text'])
polaritas = 0
status = []
total_positif_naive = total_negatif_naive = total_netral_naive = total = 0

for i, tweet in enumerate(data_tweet):
  analysis = TextBlob(tweet, classifier=cl)

  if analysis.classify() == 'positif':
    total_positif_naive += 1
  elif analysis.classify() == 'netral':
    total_netral_naive += 1
  else:
    total_negatif_naive += 1
  total += 1
  status.append(analysis.classify())

print(f'Hasil analisis data:\nPositif = {total_positif_naive}\nNetral = {total_netral_naive}\nNegatif = {total_negatif_naive}')
print(f'total data = {total}')

Hasil analisis data:
Positif = 1583
Netral = 657
Negatif = 221
total data = 2461


In [ ]:
status = pd.DataFrame({'klasifikasi bayes': status})
data['klasifikasi_bayes'] = status
data

,full_text,label,klasifikasi_bayes
0,pasti anies tidak punya prestasi,negatif,positif
1,suasana desa memang selalu senang udara segar ...,netral,netral
2,program anies hanya ubah nama rumah sakit se i...,negatif,negatif
3,anies bohong dan bohong tanpa rasa salah,negatif,positif
4,tidak bicara sesuai fakta anies dengan lantang...,negatif,negatif
...,...,...,...
2456,kata sila cek di gugel udah banyak swasta yang...,netral,netral
2457,yang penting bukan tugas partai apalagi bokeper,netral,positif
2458,solo bukti bahwa gibran punya rencana ekonomi ...,positif,positif
2459,tapi fans nya yang muslim pada tidak paling im...,netral,netral


# Word2vec

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from gensim.models import Word2Vec

sentences = data['full_text'] # List teks
tokenized_sentences = [sentence.split() for sentence in sentences]

model = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1, workers=5)
X = np.array([np.mean([model.wv[word] for word in sentence if word in model.wv.key_to_index], axis=0) for sentence in tokenized_sentences])
labels = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [ ]:
# Klasifikasi dengan KNN
param = {'weights':('uniform', 'distance'),
         'algorithm':('auto', 'ball_tree', 'kd_tree', 'brute'),
         'leaf_size':[30, 60, 90, 120],
         'p':[1,2],
         'n_neighbors':[1,2,3,4,5,6,7,8,9,10]}
KNN = KNeighborsClassifier()
knn_classifier = GridSearchCV(KNN, param)
knn_classifier.fit(X_train, y_train)
print(knn_classifier.best_params_)
knn_predictions_train = knn_classifier.predict(X_train)
knn_accuracy_train = accuracy_score(y_train, knn_predictions_train)
knn_predictions_test = knn_classifier.predict(X_test)
knn_accuracy_test = accuracy_score(y_test, knn_predictions_test)
print("== knn Accuracy ==")
print("knn Accuracy train:", knn_accuracy_train)
print("knn Accuracy test:", knn_accuracy_test)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 10, 'p': 1, 'weights': 'distance'}
== knn Accuracy ==
knn Accuracy train: 1.0
knn Accuracy test: 0.6592292089249493


In [ ]:
param = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'),
         'degree':[1,2,3,4,5,6],
         'gamma': ('scale', 'auto')}
# Klasifikasi dengan SVM
SVM = SVC()
svm_classifier = GridSearchCV(SVM, param)
svm_classifier.fit(X_train, y_train)
print(svm_classifier.best_params_)
svm_predictions_train = svm_classifier.predict(X_train)
svm_accuracy_train = accuracy_score(y_train, svm_predictions_train)
svm_predictions_test = svm_classifier.predict(X_test)
svm_accuracy_test = accuracy_score(y_test, svm_predictions_test)
print("== svm Accuracy ==")
print("svm Accuracy train:", svm_accuracy_train)
print("svm Accuracy test:", svm_accuracy_test)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
== svm Accuracy ==
svm Accuracy train: 0.676829268292683
svm Accuracy test: 0.6389452332657201


In [ ]:
#klasifikasi dengan randomforest
# param = {'n_estimators':[10,50,100,150,200,250],
#          'criterion':('gini', 'entropy', 'log_loss'),
#          'max_depth': [2,3,4,5,6,7,8,9,10],
#          'max_features': ('sqrt', 'log2')}

rfc_classifier = RandomForestClassifier(n_estimators=200, max_depth=3, criterion='gini')
rfc_classifier.fit(X_train, y_train)
rfc_predictions_train = rfc_classifier.predict(X_train)
rfc_accuracy_train = accuracy_score(y_train, rfc_predictions_train)
rfc_predictions_test = rfc_classifier.predict(X_test)
rfc_accuracy_test = accuracy_score(y_test, rfc_predictions_test)
print("== rfc Accuracy ==")
print("rfc Accuracy train:", rfc_accuracy_train)
print("rfc Accuracy test:", rfc_accuracy_test)

== rfc Accuracy ==
rfc Accuracy train: 0.649390243902439
rfc Accuracy test: 0.6206896551724138


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbr_classifier = GradientBoostingClassifier(max_depth=3, n_estimators=200)
gbr_classifier.fit(X_train, y_train)
gbr_predictions_train = gbr_classifier.predict(X_train)
gbr_accuracy_train = accuracy_score(y_train, gbr_predictions_train)
gbr_predictions_test = gbr_classifier.predict(X_test)
gbr_accuracy_test = accuracy_score(y_test, gbr_predictions_test)
print("== gbr Accuracy ==")
print("gbr Accuracy train:", gbr_accuracy_train)
print("gbr Accuracy test:", gbr_accuracy_test)

== gbr Accuracy ==
gbr Accuracy train: 0.9888211382113821
gbr Accuracy test: 0.7018255578093306


In [ ]:
from sklearn.ensemble import BaggingClassifier

bc_classifier = BaggingClassifier(n_estimators=100)
bc_classifier.fit(X_train, y_train)
bc_predictions_train = bc_classifier.predict(X_train)
bc_accuracy_train = accuracy_score(y_train, bc_predictions_train)
bc_predictions_test = bc_classifier.predict(X_test)
bc_accuracy_test = accuracy_score(y_test, bc_predictions_test)
print("== bc Accuracy ==")
print("bc Accuracy train:", bc_accuracy_train)
print("bc Accuracy test:", bc_accuracy_test)

== bc Accuracy ==
bc Accuracy train: 1.0
bc Accuracy test: 0.6774847870182555


# TF iDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(train_data['full_text'])
test_vectors = vectorizer.transform(test_data['full_text'])

In [ ]:
from sklearn.metrics import classification_report

param = {'weights':('uniform', 'distance'),
         'leaf_size':[60, 90, 120],
         'p':[1,2],
         'n_neighbors':[3,5,7]}
KNN = KNeighborsClassifier()
knn_tfidf = GridSearchCV(KNN, param)
knn_tfidf.fit(train_vectors, train_data['label'])
print(knn_tfidf.best_params_)
knn_tfidf_pred_train = knn_tfidf.predict(train_vectors)
knn_tfidf_acc_train = accuracy_score(train_data['label'], knn_tfidf_pred_train)
knn_tfidf_pred_test = knn_tfidf.predict(test_vectors)
knn_tfidf_acc_test = accuracy_score(test_data['label'], knn_tfidf_pred_test)
print("== knn accuracy ==")
print('accuracy knn train: ', knn_tfidf_acc_train)
print('accuracy knn test: ', knn_tfidf_acc_test)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'leaf_size': 60, 'n_neighbors': 3, 'p': 2, 'weights': 'distance'}
== knn accuracy ==
accuracy knn train:  0.9989837398373984
accuracy knn test:  0.4847870182555781


In [ ]:
param = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'),
         'degree':[1,2,3,4,5,6],
         'gamma': ('scale', 'auto')}
SVM = SVC()
svm_tfidf = GridSearchCV(SVM, param)
svm_tfidf.fit(train_vectors, train_data['label'])
print(svm_tfidf.best_params_)
svm_tfidf_pred_train = svm_tfidf.predict(train_vectors)
svm_tfidf_acc_train = accuracy_score(train_data['label'], svm_tfidf_pred_train)
svm_tfidf_pred_test = svm_tfidf.predict(test_vectors)
svm_tfidf_acc_test = accuracy_score(test_data['label'], svm_tfidf_pred_test)
print("== svm accuracy ==")
print('accuracy svm train: ', svm_tfidf_acc_train)
print('accuracy svm test: ', svm_tfidf_acc_test)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
== svm accuracy ==
accuracy svm train:  0.9496951219512195
accuracy svm test:  0.7099391480730223


In [ ]:
rfc_tfidf = RandomForestClassifier(n_estimators=200, criterion='entropy')
rfc_tfidf.fit(train_vectors, train_data['label'])
rfc_tfidf_pred_train = rfc_tfidf.predict(train_vectors)
rfc_tfidf_acc_train = accuracy_score(train_data['label'], rfc_tfidf_pred_train)
rfc_tfidf_pred_test = rfc_tfidf.predict(test_vectors)
rfc_tfidf_acc_test = accuracy_score(test_data['label'], rfc_tfidf_pred_test)
print("== rfc accuracy ==")
print('accuracy rfc train: ', rfc_tfidf_acc_train)
print('accuracy rfc test: ', rfc_tfidf_acc_test)

== rfc accuracy ==
accuracy rfc train:  0.9989837398373984
accuracy rfc test:  0.6997971602434077


# Count Vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

X_train, X_test, y_train, y_test = train_test_split(data['full_text'], data['label'], random_state=42, test_size=0.2)

vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)


In [ ]:
param = {'weights':('uniform', 'distance'),
         'leaf_size':[60, 90, 120],
         'p':[1,2],
         'n_neighbors':[3,5,7]}
KNN = KNeighborsClassifier()
knn_cv = GridSearchCV(KNN, param)
knn_cv.fit(X_train_vectorized, y_train)
print(knn_cv.best_params_)
knn_cv_pred_train = knn_cv.predict(X_train_vectorized)
knn_cv_train_acc = accuracy_score(y_train, knn_cv_pred_train)
knn_cv_pred_test = knn_cv.predict(X_test_vectorized)
knn_cv_test_acc = accuracy_score(y_test, knn_cv_pred_test)
print('== knn accuracy ==')
print('accuracy knn train: ', knn_cv_train_acc)
print('accuracy knn test: ', knn_cv_test_acc)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'leaf_size': 60, 'n_neighbors': 3, 'p': 2, 'weights': 'distance'}
== knn accuracy ==
accuracy knn train:  1.0
accuracy knn test:  0.6612576064908722


In [ ]:
param = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'),
         'degree':[1,2,3,4,5,6],
         'gamma': ('scale', 'auto')}
SVM = SVC()
svm_cv = GridSearchCV(SVM, param)
svm_cv.fit(X_train_vectorized, y_train)
print(svm_cv.best_params_)
svm_cv_pred_train = svm_cv.predict(X_train_vectorized)
svm_cv_train_acc = accuracy_score(y_train, svm_cv_pred_train)
svm_cv_pred_test = svm_cv.predict(X_test_vectorized)
svm_cv_test_acc = accuracy_score(y_test, svm_cv_pred_test)
print('== svm accuracy ==')
print('accuracy svm train: ', svm_cv_train_acc)
print('accuracy svm test: ', svm_cv_test_acc)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
== svm accuracy ==
accuracy svm train:  0.926829268292683
accuracy svm test:  0.718052738336714


In [ ]:
from sklearn import svm
from sklearn.svm import SVR
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV

# Create a synthetic dataset for demonstration purposes
X, y = make_classification(n_samples=100, n_features=20, n_informative=2, n_redundant=10, random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 1]}

# Instantiate the SVR model with L1 regularization
svr_model = SVR(kernel='linear', C=1, epsilon=0.1, tol=1e-3)

# Perform GridSearchCV
svr_cv = GridSearchCV(svr_model, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
svr_cv.fit(X, y)

# Print the best parameters and accuracy
print('Best parameters:', svr_cv.best_params_)
print('Best accuracy:', svr_cv.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best parameters: {'C': 0.1, 'epsilon': 0.01}
Best accuracy: nan


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [ ]:
rfc_cv = RandomForestClassifier(n_estimators=200, criterion='gini', max_depth=12)
rfc_cv.fit(X_train_vectorized, y_train)
rfc_cv_pred_train = rfc_cv.predict(X_train_vectorized)
rfc_cv_train_acc = accuracy_score(y_train, rfc_cv_pred_train)
rfc_cv_pred_test = rfc_cv.predict(X_test_vectorized)
rfc_cv_test_acc = accuracy_score(y_test, rfc_cv_pred_test)
print('== rfc accuracy ==')
print('accuracy rfc train: ', rfc_cv_train_acc)
print('accuracy rfc test: ', rfc_cv_test_acc)

== rfc accuracy ==
accuracy rfc train:  0.6473577235772358
accuracy rfc test:  0.6206896551724138


# gloVe

In [ ]:
!pip install gensim

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
import gensim.downloader as api

glove_model = api.load("glove-wiki-gigaword-100")

glove_vectors = {}
for word in glove_model.key_to_index.keys():
    glove_vectors[word] = glove_model[word]


def sentence_to_vec(sentence):
    words = sentence.split()
    vectors = [glove_vectors[word] for word in words if word in glove_vectors]
    return np.mean(vectors, axis=0) if vectors else np.zeros_like(glove_vectors[next(iter(glove_vectors))])


X = np.array([sentence_to_vec(sentence) for sentence in data['full_text']])
y = data['label']

# Pisahkan data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
param = {'weights':('uniform', 'distance'),
         'leaf_size':[60, 90, 120],
         'p':[1,2],
         'n_neighbors':[3,5,7]}
KNN = KNeighborsClassifier()
knn_gv = GridSearchCV(KNN, param)
knn_gv.fit(X_train, y_train)
print(knn_gv.best_params_)
knn_gv_pred_train = knn_gv.predict(X_train)
knn_gv_train_acc = accuracy_score(y_train, knn_gv_pred_train)
knn_gv_pred_test = knn_gv.predict(X_test)
knn_gv_test_acc = accuracy_score(y_test, knn_gv_pred_test)
print('== knn accuracy ==')
print('accuracy knn train: ', knn_gv_train_acc)
print('accuracy knn test: ', knn_gv_test_acc)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'leaf_size': 60, 'n_neighbors': 7, 'p': 2, 'weights': 'distance'}
== knn accuracy ==
accuracy knn train:  0.9994918699186992
accuracy knn test:  0.6612576064908722


In [ ]:
param = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'),
         'degree':[1,2,3,4,5,6],
         'gamma': ('scale', 'auto')}
SVM = SVC()
svm_gv = GridSearchCV(SVM, param)
svm_gv.fit(X_train, y_train)
print(svm_gv.best_params_)
svm_gv_pred_train = svm_gv.predict(X_train)
svm_gv_train_acc = accuracy_score(y_train, svm_gv_pred_train)
svm_gv_pred_test = svm_gv.predict(X_test)
svm_gv_test_acc = accuracy_score(y_test, svm_gv_pred_test)
print('== svm accuracy ==')
print('accuracy svm train: ', svm_gv_train_acc)
print('accuracy svm test: ', svm_gv_test_acc)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
== svm accuracy ==
accuracy svm train:  0.771849593495935
accuracy svm test:  0.6592292089249493


In [ ]:
rfc_gv = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=3)
rfc_gv.fit(X_train, y_train)
rfc_gv_pred_train = rfc_gv.predict(X_train)
rfc_gv_train_acc = accuracy_score(y_train, rfc_gv_pred_train)
rfc_gv_pred_test = rfc_gv.predict(X_test)
rfc_gv_test_acc = accuracy_score(y_test, rfc_gv_pred_test)
print('== rfc accuracy ==')
print('accuracy rfc train: ', rfc_gv_train_acc)
print('accuracy rfc test: ', rfc_gv_test_acc)

== rfc accuracy ==
accuracy rfc train:  0.6483739837398373
accuracy rfc test:  0.6247464503042597


# Anies

In [ ]:
anies = pd.read_csv('preProcessingAnies.csv')
anies

,full_text,label
0,pasti anies tidak punya prestasi,negatif
1,suasana desa memang selalu senang udara segar ...,netral
2,program anies hanya ubah nama rumah sakit se i...,negatif
3,anies bohong dan bohong tanpa rasa salah,negatif
4,tidak bicara sesuai fakta anies dengan lantang...,negatif
...,...,...
483,tiga untuk indonesia lebih baik tag batter up ...,netral
484,makanya susah paham apa yang disampaikam anies...,negatif
485,buat pak anies bukankah anda saat itu yang kd ...,negatif
486,anies baswedan sosok pimpin yang paham bahwa p...,positif


In [ ]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

sentences = anies['full_text'] # List teks
tokenized_sentences = [sentence.split() for sentence in sentences]

model = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1, workers=5)
X = np.array([np.mean([model.wv[word] for word in sentence if word in model.wv.key_to_index], axis=0) for sentence in tokenized_sentences])
labels = anies['label']

X_train_anies, X_test_anies, y_train_anies, y_test_anies = train_test_split(X, labels, test_size=0.6, random_state=42)

In [ ]:
knn_predictions_test = knn_classifier.predict(X_test_anies)
knn_accuracy_test = accuracy_score(y_test_anies, knn_predictions_test)
print("== knn Accuracy ==")
print("knn Accuracy test:", knn_accuracy_test)

svm_predictions_test = svm_classifier.predict(X_test_anies)
svm_accuracy_test = accuracy_score(y_test_anies, svm_predictions_test)
print("== svm Accuracy ==")
print("svm Accuracy test:", svm_accuracy_test)

rfc_predictions_test = rfc_classifier.predict(X_test_anies)
rfc_accuracy_test = accuracy_score(y_test_anies, rfc_predictions_test)
print("== rfc Accuracy ==")
print("rfc Accuracy test:", rfc_accuracy_test)

== knn Accuracy ==
knn Accuracy test: 0.39590443686006827
== svm Accuracy ==
svm Accuracy test: 0.39590443686006827
== rfc Accuracy ==
rfc Accuracy test: 0.39590443686006827


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors_anies = vectorizer.fit_transform(anies['full_text'])
test_vectors_anies = vectorizer.transform(anies['full_text'])

In [ ]:
knn_tfidf_pred_test = knn_tfidf.predict(test_vectors_anies)
knn_tfidf_acc_test = accuracy_score(anies['label'], knn_tfidf_pred_test)
print("== knn accuracy ==")
print('accuracy knn test: ', knn_tfidf_acc_test)

ValueError: X has 413 features, but KNeighborsClassifier is expecting 964 features as input.

In [ ]:
# Membuat dictionary glove_vectors
glove_vectors = {}
for word in glove_model.key_to_index.keys():
    glove_vectors[word] = glove_model[word]

# Fungsi untuk mengonversi kalimat menjadi vektor rata-rata GloVe
def sentence_to_vec(sentence):
    words = sentence.split()
    vectors = [glove_vectors[word] for word in words if word in glove_vectors]
    return np.mean(vectors, axis=0) if vectors else np.zeros_like(glove_vectors[next(iter(glove_vectors))])

# Mengonversi data teks ke vektor GloVe
X = np.array([sentence_to_vec(sentence) for sentence in anies['full_text']])
y = anies['label']

# Pisahkan data
X_train_anies, X_test_anies, y_train_anies, y_test_anies = train_test_split(X, y, test_size=1, random_state=42)

In [ ]:
anies_gv_pred_knn = knn_gv.predict(X_test_anies)
anies_gv_acc_knn = accuracy_score(y_test_anies, anies_gv_pred_knn)
print('== knn accuracy ==')
print('accuracy knn anies: ', anies_gv_acc_knn)

anies_gv_pred_svm = svm_gv.predict(X_test_anies)
anies_gv_acc_svm = accuracy_score(y_test_anies, anies_gv_pred_svm)
print('== svm accuracy ==')
print('accuracy svm anies: ', anies_gv_acc_svm)

anies_gv_pred_rfc = rfc_gv.predict(X_test_anies)
anies_gv_acc_rfc = accuracy_score(y_test_anies, anies_gv_pred_rfc)
print('== rfc accuracy ==')
print('accuracy rfc anies: ', anies_gv_acc_rfc)

== knn accuracy ==
accuracy knn anies:  1.0
== svm accuracy ==
accuracy svm anies:  1.0
== rfc accuracy ==
accuracy rfc anies:  1.0
